In [22]:
import pandas as pd
import json
import sqlite3

# 1. Load and Merge Data
orders_df = pd.read_csv('orders.csv')
with open('users.json', 'r') as f:
    users_df = pd.DataFrame(json.load(f))

conn = sqlite3.connect(':memory:')
with open('restaurants.sql', 'r') as f:
    conn.executescript(f.read())
restaurants_df = pd.read_sql_query("SELECT * FROM restaurants", conn)

merged_df = pd.merge(orders_df, users_df, on='user_id', how='left')
final_df = pd.merge(merged_df, restaurants_df, on='restaurant_id', how='left', suffixes=('', '_sql'))
final_df['order_date'] = pd.to_datetime(final_df['order_date'], dayfirst=True)
final_df['quarter'] = final_df['order_date'].dt.quarter

# 2. Reference Table: City & Membership
print("CITY & MEMBERSHIP ")
city_ref = final_df.groupby(['city', 'membership'])['total_amount'].agg(
    Total_Revenue='sum',
    Average_Order_Value='mean',
    Order_Count='count'
).reset_index()
print(city_ref)

# 3. Reference Table: Cuisine & Membership
print("\nCUISINE & MEMBERSHIP")
cuisine_ref = final_df.groupby(['cuisine', 'membership']).agg(
    Total_Revenue=('total_amount', 'sum'),
    Average_Order_Value=('total_amount', 'mean'),
    Distinct_Restaurants=('restaurant_id', 'nunique')
).reset_index()
print(cuisine_ref)

# 4. Reference Table: Rating & Time
print("\nRATINGS & QUARTERS")
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ['3.0-3.5', '3.6-4.0', '4.1-4.5', '4.6-5.0']
final_df['rating_range'] = pd.cut(final_df['rating'], bins=bins, labels=labels, include_lowest=True)

rating_summary = final_df.groupby('rating_range', observed=True)['total_amount'].sum()
quarter_summary = final_df.groupby('quarter')['total_amount'].sum()
print("Revenue by Rating Range:\n", rating_summary)
print("\nRevenue by Quarter:\n", quarter_summary)

# 5. Reference Table: User & Restaurant Specifics
print("\n USER & RESTAURANT STATS")
user_spend = final_df.groupby('user_id')['total_amount'].sum()
print(f"Total Distinct Users: {final_df['user_id'].nunique()}")
print(f"Users who spent > 1000 total: {(user_spend > 1000).sum()}")

# Added: Total row count of the final merged dataset
print(f"Total rows in the final dataset: {len(final_df)}")

rest_ref = final_df.groupby('restaurant_name')['total_amount'].agg(
    Avg_Order='mean',
    Order_Count='count'
)
options = ['Grand Cafe Punjabi', 'Grand Restaurant South Indian', 'Ruchi Mess Multicuisine', 'Ruchi Foods Chinese']
print("\nSpecific Restaurant Check:\n", rest_ref[rest_ref.index.isin(options)])

# 6. Final Dataset Table
print("\nFINAL DATASET PREVIEW")
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
final_df.head(10)

CITY & MEMBERSHIP 
        city membership  Total_Revenue  Average_Order_Value  Order_Count
0  Bangalore       Gold      994702.59           793.223756         1254
1  Bangalore    Regular     1212243.99           809.782224         1497
2    Chennai       Gold     1080909.79           808.459080         1337
3    Chennai    Regular      909603.24           803.536431         1132
4  Hyderabad       Gold      896740.19           806.421034         1112
5  Hyderabad    Regular      992626.39           801.798376         1238
6       Pune       Gold     1003012.32           781.162243         1284
7       Pune    Regular      921785.61           804.350445         1146

CUISINE & MEMBERSHIP
   cuisine membership  Total_Revenue  Average_Order_Value  \
0  Chinese       Gold      977713.74           805.365519   
1  Chinese    Regular      952790.91           791.354576   
2   Indian       Gold      979312.31           792.323875   
3   Indian    Regular      992100.27           804.623090 

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name,name,city,membership,restaurant_name_sql,cuisine,rating,quarter,rating_range
0,1,2508,450,2023-02-18,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2,1,3.0-3.5
1,2,2693,309,2023-01-18,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5,1,4.1-4.5
2,3,2084,107,2023-07-15,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0,3,3.6-4.0
3,4,319,224,2023-10-04,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8,4,4.6-5.0
4,5,1064,293,2023-12-25,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0,4,3.0-3.5
5,6,2933,499,2023-07-12,1497.22,Annapurna Tiffins South Indian,User_2933,Chennai,Regular,Restaurant_499,Mexican,4.9,3,4.6-5.0
6,7,970,35,2023-05-30,129.21,Royal Biryani North Indian,User_970,Bangalore,Regular,Restaurant_35,Chinese,3.9,2,3.6-4.0
7,8,891,57,2023-11-07,269.19,Spice Mess Punjabi,User_891,Pune,Regular,Restaurant_57,Mexican,4.4,4,4.1-4.5
8,9,364,7,2023-12-05,953.30,Ruchi Biryani Punjabi,User_364,Chennai,Gold,Restaurant_7,Italian,3.2,4,3.0-3.5
9,10,2972,183,2023-12-30,351.41,Taste of Biryani Non-Veg,User_2972,Bangalore,Regular,Restaurant_183,Chinese,3.7,4,3.6-4.0
